# 📊 Chapitre 07 : Probabilités Fondamentales

## 🎯 Objectifs du chapitre

Les probabilités sont **ABSOLUMENT ESSENTIELLES** pour le Machine Learning et la finance quantitative :

- 🤖 **Machine Learning** : Bayesian ML, Naive Bayes, échantillonnage, incertitude
- 📈 **Finance Quantitative** : Modélisation des rendements, Value at Risk, options pricing
- 🧠 **Deep Learning** : Dropout, variational inference, uncertainty quantification
- 📊 **Statistics** : Fondation de tous les tests statistiques et modèles

### Ce que vous allez maîtriser :

1. ✅ Espaces de probabilité et événements
2. ✅ Probabilités conditionnelles et indépendance
3. ✅ **Théorème de Bayes** (crucial en ML !)
4. ✅ Variables aléatoires discrètes et continues
5. ✅ Distributions de probabilité (Bernoulli, Binomiale, Poisson, Normale)
6. ✅ Espérance, variance, moments
7. ✅ Covariance et corrélation

---

In [ ]:
# Imports nécessaires
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.special import comb
import pandas as pd
from typing import List, Tuple

# Configuration des graphiques
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# Seed pour reproductibilité
np.random.seed(42)

## 1️⃣ Espace de Probabilité et Événements

### 📚 Théorie

Un **espace de probabilité** est un triplet $(\Omega, \mathcal{F}, P)$ où :

- $\Omega$ : **Espace échantillonnal** (ensemble de tous les résultats possibles)
- $\mathcal{F}$ : **Tribu** ou σ-algèbre (ensemble des événements mesurables)
- $P$ : **Mesure de probabilité** qui satisfait :
  - $P(\Omega) = 1$
  - $P(A) \geq 0$ pour tout $A \in \mathcal{F}$
  - Si $A_1, A_2, ...$ sont disjoints : $P(\bigcup_{i=1}^{\infty} A_i) = \sum_{i=1}^{\infty} P(A_i)$

### 🎲 Opérations sur les événements

- **Union** : $A \cup B$ (A ou B)
- **Intersection** : $A \cap B$ (A et B)
- **Complémentaire** : $A^c$ (non A)
- **Différence** : $A \setminus B$ = $A \cap B^c$

### 📐 Axiomes de Kolmogorov

$$P(A \cup B) = P(A) + P(B) - P(A \cap B)$$
$$P(A^c) = 1 - P(A)$$
$$P(\emptyset) = 0$$

In [ ]:
# Exemple : Lancer de dé
class EspaceProbabilite:
    """Représentation d'un espace de probabilité discret"""
    
    def __init__(self, omega: List, probas: List[float]):
        """
        omega: Liste des résultats possibles
        probas: Liste des probabilités correspondantes
        """
        assert len(omega) == len(probas), "Dimensions incompatibles"
        assert abs(sum(probas) - 1.0) < 1e-10, "Les probabilités doivent sommer à 1"
        
        self.omega = omega
        self.probas = np.array(probas)
        self.proba_dict = dict(zip(omega, probas))
    
    def P(self, evenement: List) -> float:
        """Calcule P(événement)"""
        return sum(self.proba_dict.get(e, 0) for e in evenement)
    
    def union(self, A: List, B: List) -> List:
        """A ∪ B"""
        return list(set(A) | set(B))
    
    def intersection(self, A: List, B: List) -> List:
        """A ∩ B"""
        return list(set(A) & set(B))
    
    def complementaire(self, A: List) -> List:
        """A^c"""
        return [e for e in self.omega if e not in A]

# Exemple : Dé équilibré
de = EspaceProbabilite(
    omega=[1, 2, 3, 4, 5, 6],
    probas=[1/6] * 6
)

# Événements
A = [2, 4, 6]  # Nombre pair
B = [1, 2, 3]  # Nombre ≤ 3

print("🎲 Espace de probabilité : Lancer de dé\n")
print(f"P(nombre pair) = P({A}) = {de.P(A):.4f}")
print(f"P(nombre ≤ 3) = P({B}) = {de.P(B):.4f}")
print(f"\nP(A ∪ B) = {de.P(de.union(A, B)):.4f}")
print(f"P(A ∩ B) = {de.P(de.intersection(A, B)):.4f}")
print(f"P(A^c) = {de.P(de.complementaire(A)):.4f}")
print(f"\nVérification : P(A) + P(A ∩ B) - P(A ∩ B) = {de.P(A) + de.P(B) - de.P(de.intersection(A, B)):.4f}")

## 2️⃣ Probabilités Conditionnelles

### 📚 Définition

La probabilité de $A$ **sachant** $B$ (notée $P(A|B)$) est :

$$P(A|B) = \frac{P(A \cap B)}{P(B)}$$

si $P(B) > 0$.

### 🔗 Formule de multiplication

$$P(A \cap B) = P(A|B) \cdot P(B) = P(B|A) \cdot P(A)$$

### 🎯 Indépendance

Deux événements $A$ et $B$ sont **indépendants** si :

$$P(A \cap B) = P(A) \cdot P(B)$$

Équivalent à : $P(A|B) = P(A)$ (l'information sur $B$ ne change pas $A$)

### 🌳 Formule des probabilités totales

Si $(B_1, ..., B_n)$ forme une partition de $\Omega$ :

$$P(A) = \sum_{i=1}^{n} P(A|B_i) \cdot P(B_i)$$

In [ ]:
# Exemple : Test médical
def probabilite_conditionnelle(p_a_inter_b: float, p_b: float) -> float:
    """Calcule P(A|B) = P(A∩B) / P(B)"""
    if p_b == 0:
        raise ValueError("P(B) ne peut pas être 0")
    return p_a_inter_b / p_b

# Problème : Test de dépistage
# M = malade, T = test positif
P_M = 0.01  # 1% de la population est malade
P_T_sachant_M = 0.95  # Sensibilité : 95% de vrais positifs
P_T_sachant_non_M = 0.05  # 5% de faux positifs

# Calcul de P(T) par probabilités totales
P_T = P_T_sachant_M * P_M + P_T_sachant_non_M * (1 - P_M)

print("🏥 Exemple : Test médical\n")
print(f"Prévalence de la maladie : {P_M*100:.1f}%")
print(f"Sensibilité du test : {P_T_sachant_M*100:.1f}%")
print(f"Taux de faux positifs : {P_T_sachant_non_M*100:.1f}%")
print(f"\nP(Test positif) = {P_T:.4f}")

# Visualisation
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Diagramme en barres
categories = ['Malades\nTestés+', 'Sains\nTestés+', 'Malades\nTestés-', 'Sains\nTestés-']
probas = [
    P_T_sachant_M * P_M,
    P_T_sachant_non_M * (1 - P_M),
    (1 - P_T_sachant_M) * P_M,
    (1 - P_T_sachant_non_M) * (1 - P_M)
]
colors = ['green', 'orange', 'red', 'blue']

ax1.bar(categories, probas, color=colors, alpha=0.7, edgecolor='black')
ax1.set_ylabel('Probabilité')
ax1.set_title('Distribution des cas')
ax1.grid(axis='y', alpha=0.3)

for i, (cat, prob) in enumerate(zip(categories, probas)):
    ax1.text(i, prob + 0.01, f'{prob:.4f}', ha='center', va='bottom')

# Diagramme de Venn
from matplotlib.patches import Circle
ax2.set_xlim(0, 4)
ax2.set_ylim(0, 4)
ax2.set_aspect('equal')

circle_M = Circle((1.5, 2), 1, color='red', alpha=0.3, label='Malades')
circle_T = Circle((2.5, 2), 1, color='blue', alpha=0.3, label='Test+')
ax2.add_patch(circle_M)
ax2.add_patch(circle_T)

ax2.text(1.2, 2, f'M\n{P_M:.3f}', ha='center', va='center', fontsize=10)
ax2.text(2.8, 2, f'T\n{P_T:.3f}', ha='center', va='center', fontsize=10)
ax2.text(2, 2, f'M∩T\n{P_T_sachant_M * P_M:.4f}', ha='center', va='center', fontsize=9)

ax2.set_title('Diagramme de Venn')
ax2.legend()
ax2.axis('off')

plt.tight_layout()
plt.show()

## 3️⃣ Théorème de Bayes 🌟

### 🎓 LE théorème fondamental du ML !

$$P(A|B) = \frac{P(B|A) \cdot P(A)}{P(B)}$$

Ou, en version complète avec probabilités totales :

$$P(A_i|B) = \frac{P(B|A_i) \cdot P(A_i)}{\sum_{j=1}^{n} P(B|A_j) \cdot P(A_j)}$$

### 📖 Vocabulaire bayésien

- $P(A)$ : **Prior** (probabilité a priori)
- $P(B|A)$ : **Likelihood** (vraisemblance)
- $P(A|B)$ : **Posterior** (probabilité a posteriori)
- $P(B)$ : **Evidence** (normalisation)

### 🤖 ESSENTIEL EN ML !

Le théorème de Bayes est la base de :
- **Naive Bayes classifier**
- **Bayesian inference**
- **Bayesian optimization**
- **Probabilistic graphical models**
- **Posterior inference in neural networks**

In [ ]:
def theoreme_bayes(prior: float, likelihood: float, evidence: float) -> float:
    """Calcule le posterior avec le théorème de Bayes"""
    return (likelihood * prior) / evidence

# Retour sur le test médical avec Bayes
print("🧬 Théorème de Bayes : Test médical\n")

# P(Malade | Test+) = ?
prior = P_M  # P(Malade)
likelihood = P_T_sachant_M  # P(Test+ | Malade)
evidence = P_T  # P(Test+)

posterior = theoreme_bayes(prior, likelihood, evidence)

print(f"Prior P(Malade) = {prior:.4f}")
print(f"Likelihood P(Test+|Malade) = {likelihood:.4f}")
print(f"Evidence P(Test+) = {evidence:.4f}")
print(f"\n🎯 Posterior P(Malade|Test+) = {posterior:.4f}")
print(f"\n💡 Si test positif, seulement {posterior*100:.1f}% de chances d'être malade!")
print(f"   (Car la prévalence est faible et il y a des faux positifs)")

# Exemple 2 : Classification bayésienne
print("\n" + "="*60)
print("📧 Exemple : Filtre anti-spam bayésien\n")

# Données
P_spam = 0.3  # 30% des emails sont des spams
P_mot_viagra_sachant_spam = 0.8  # 80% des spams contiennent "viagra"
P_mot_viagra_sachant_ham = 0.05  # 5% des emails légitimes contiennent "viagra"

# P("viagra") par probabilités totales
P_mot_viagra = (P_mot_viagra_sachant_spam * P_spam + 
                P_mot_viagra_sachant_ham * (1 - P_spam))

# P(Spam | "viagra") par Bayes
P_spam_sachant_viagra = theoreme_bayes(
    prior=P_spam,
    likelihood=P_mot_viagra_sachant_spam,
    evidence=P_mot_viagra
)

print(f"Prior P(Spam) = {P_spam:.2f}")
print(f"P('viagra'|Spam) = {P_mot_viagra_sachant_spam:.2f}")
print(f"P('viagra'|Ham) = {P_mot_viagra_sachant_ham:.2f}")
print(f"\n🎯 P(Spam|'viagra') = {P_spam_sachant_viagra:.4f}")
print(f"\n💡 Si email contient 'viagra', {P_spam_sachant_viagra*100:.1f}% de chances que ce soit un spam!")

In [ ]:
# Visualisation de l'update bayésien
def visualiser_update_bayesien(priors: np.ndarray, likelihoods: np.ndarray, 
                               labels: List[str], titre: str):
    """
    Visualise comment le prior devient posterior après observation
    """
    # Calcul des posteriors
    evidence = np.sum(priors * likelihoods)
    posteriors = (likelihoods * priors) / evidence
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # Prior
    axes[0].bar(labels, priors, color='steelblue', alpha=0.7, edgecolor='black')
    axes[0].set_title('Prior P(H)', fontsize=12, fontweight='bold')
    axes[0].set_ylabel('Probabilité')
    axes[0].set_ylim(0, 1)
    axes[0].grid(axis='y', alpha=0.3)
    
    # Likelihood
    axes[1].bar(labels, likelihoods, color='orange', alpha=0.7, edgecolor='black')
    axes[1].set_title('Likelihood P(D|H)', fontsize=12, fontweight='bold')
    axes[1].set_ylabel('Probabilité')
    axes[1].set_ylim(0, 1)
    axes[1].grid(axis='y', alpha=0.3)
    
    # Posterior
    axes[2].bar(labels, posteriors, color='green', alpha=0.7, edgecolor='black')
    axes[2].set_title('Posterior P(H|D)', fontsize=12, fontweight='bold')
    axes[2].set_ylabel('Probabilité')
    axes[2].set_ylim(0, 1)
    axes[2].grid(axis='y', alpha=0.3)
    
    # Annotations
    for ax, values in zip(axes, [priors, likelihoods, posteriors]):
        for i, v in enumerate(values):
            ax.text(i, v + 0.02, f'{v:.3f}', ha='center', va='bottom', fontweight='bold')
    
    plt.suptitle(titre, fontsize=14, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()
    
    return posteriors

# Exemple : Diagnostic médical avec 3 maladies possibles
labels = ['Grippe', 'Covid', 'Allergie']
priors = np.array([0.5, 0.1, 0.4])  # Prévalences
likelihoods = np.array([0.3, 0.9, 0.1])  # P(fièvre | maladie)

posteriors = visualiser_update_bayesien(
    priors, likelihoods, labels,
    "Update Bayésien : Diagnostic avec symptôme 'fièvre'"
)

print("\n🔄 Mise à jour bayésienne :")
for label, prior, post in zip(labels, priors, posteriors):
    print(f"{label:10s} : {prior:.3f} → {post:.3f} (×{post/prior:.2f})")

## 4️⃣ Variables Aléatoires

### 📚 Définition

Une **variable aléatoire** (VA) est une fonction $X: \Omega \to \mathbb{R}$ qui associe un nombre réel à chaque résultat.

### 🎲 Variables aléatoires discrètes

Une VA discrète prend un nombre fini ou dénombrable de valeurs.

**Fonction de masse de probabilité (PMF)** :
$$p_X(x) = P(X = x)$$

**Propriétés** :
- $p_X(x) \geq 0$ pour tout $x$
- $\sum_{x} p_X(x) = 1$

**Fonction de répartition (CDF)** :
$$F_X(x) = P(X \leq x) = \sum_{t \leq x} p_X(t)$$

### 📊 Variables aléatoires continues

Une VA continue peut prendre toute valeur dans un intervalle.

**Fonction de densité de probabilité (PDF)** :
$$P(a \leq X \leq b) = \int_a^b f_X(x) dx$$

**Propriétés** :
- $f_X(x) \geq 0$ pour tout $x$
- $\int_{-\infty}^{\infty} f_X(x) dx = 1$
- $P(X = x) = 0$ pour toute valeur $x$ (!)  

**Fonction de répartition (CDF)** :
$$F_X(x) = P(X \leq x) = \int_{-\infty}^x f_X(t) dt$$

In [ ]:
# Exemple de VA discrète : Lancer de dé
class VariableAleatoireDiscrete:
    """Représentation d'une variable aléatoire discrète"""
    
    def __init__(self, valeurs: np.ndarray, probas: np.ndarray):
        assert len(valeurs) == len(probas)
        assert abs(np.sum(probas) - 1.0) < 1e-10
        
        self.valeurs = valeurs
        self.probas = probas
    
    def pmf(self, x: float) -> float:
        """Fonction de masse P(X=x)"""
        idx = np.where(self.valeurs == x)[0]
        return self.probas[idx[0]] if len(idx) > 0 else 0.0
    
    def cdf(self, x: float) -> float:
        """Fonction de répartition P(X≤x)"""
        return np.sum(self.probas[self.valeurs <= x])
    
    def esperance(self) -> float:
        """Espérance E[X]"""
        return np.sum(self.valeurs * self.probas)
    
    def variance(self) -> float:
        """Variance Var(X)"""
        E_X = self.esperance()
        return np.sum((self.valeurs - E_X)**2 * self.probas)
    
    def simuler(self, n: int) -> np.ndarray:
        """Génère n échantillons"""
        return np.random.choice(self.valeurs, size=n, p=self.probas)

# Exemple : Dé truqué
de_truque = VariableAleatoireDiscrete(
    valeurs=np.array([1, 2, 3, 4, 5, 6]),
    probas=np.array([0.1, 0.1, 0.1, 0.2, 0.2, 0.3])  # Favorise 6
)

# Visualisation PMF et CDF
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# PMF
ax1.bar(de_truque.valeurs, de_truque.probas, color='steelblue', 
        alpha=0.7, edgecolor='black', width=0.6)
ax1.set_xlabel('Valeur x')
ax1.set_ylabel('P(X = x)')
ax1.set_title('PMF : Fonction de masse', fontweight='bold')
ax1.set_xticks(de_truque.valeurs)
ax1.grid(axis='y', alpha=0.3)

for x, p in zip(de_truque.valeurs, de_truque.probas):
    ax1.text(x, p + 0.01, f'{p:.2f}', ha='center', va='bottom')

# CDF
x_range = np.linspace(0, 7, 100)
cdf_values = [de_truque.cdf(x) for x in x_range]

ax2.plot(x_range, cdf_values, 'b-', linewidth=2)
ax2.scatter(de_truque.valeurs, [de_truque.cdf(x) for x in de_truque.valeurs],
           color='red', s=50, zorder=5)
ax2.set_xlabel('Valeur x')
ax2.set_ylabel('P(X ≤ x)')
ax2.set_title('CDF : Fonction de répartition', fontweight='bold')
ax2.grid(alpha=0.3)
ax2.set_ylim(-0.05, 1.05)

plt.tight_layout()
plt.show()

print(f"\n📊 Caractéristiques du dé truqué :")
print(f"E[X] = {de_truque.esperance():.4f}")
print(f"Var(X) = {de_truque.variance():.4f}")
print(f"σ(X) = {np.sqrt(de_truque.variance()):.4f}")

## 5️⃣ Distributions de Probabilité Classiques

### 🎲 Distributions Discrètes

#### 1. Loi de Bernoulli : $X \sim \text{Ber}(p)$

Modélise une expérience à 2 résultats (succès/échec).

- $X \in \{0, 1\}$
- $P(X = 1) = p$, $P(X = 0) = 1-p$
- $E[X] = p$
- $\text{Var}(X) = p(1-p)$

**Utilisation** : Classification binaire, dropout en neural networks

#### 2. Loi Binomiale : $X \sim \mathcal{B}(n, p)$

Nombre de succès en $n$ essais indépendants de Bernoulli.

- $X \in \{0, 1, ..., n\}$
- $P(X = k) = \binom{n}{k} p^k (1-p)^{n-k}$
- $E[X] = np$
- $\text{Var}(X) = np(1-p)$

**Utilisation** : Taux de conversion, A/B testing

#### 3. Loi de Poisson : $X \sim \text{Poisson}(\lambda)$

Nombre d'événements rares en un temps fixé.

- $X \in \{0, 1, 2, ...\}$
- $P(X = k) = \frac{\lambda^k e^{-\lambda}}{k!}$
- $E[X] = \lambda$
- $\text{Var}(X) = \lambda$

**Utilisation** : Comptage d'événements (clics, transactions, défaillances)

### 📊 Distributions Continues

#### 4. Loi Normale (Gaussienne) : $X \sim \mathcal{N}(\mu, \sigma^2)$

**LA** distribution la plus importante en statistiques et ML !

- $f_X(x) = \frac{1}{\sigma\sqrt{2\pi}} e^{-\frac{1}{2}\left(\frac{x-\mu}{\sigma}\right)^2}$
- $E[X] = \mu$
- $\text{Var}(X) = \sigma^2$

**Propriétés remarquables** :
- Symétrique autour de $\mu$
- 68% des valeurs dans $[\mu - \sigma, \mu + \sigma]$
- 95% dans $[\mu - 2\sigma, \mu + 2\sigma]$
- 99.7% dans $[\mu - 3\sigma, \mu + 3\sigma]$

**Utilisation** : Modélisation des erreurs, rendements financiers, processus gaussiens, initialisation des poids en DL

In [ ]:
# Visualisation des distributions classiques
fig, axes = plt.subplots(2, 3, figsize=(16, 10))

# 1. Bernoulli
p = 0.3
x_bern = [0, 1]
pmf_bern = [1-p, p]
axes[0, 0].bar(x_bern, pmf_bern, color='steelblue', alpha=0.7, edgecolor='black', width=0.3)
axes[0, 0].set_title(f'Bernoulli(p={p})', fontweight='bold')
axes[0, 0].set_xlabel('x')
axes[0, 0].set_ylabel('P(X=x)')
axes[0, 0].set_xticks([0, 1])
axes[0, 0].grid(axis='y', alpha=0.3)

# 2. Binomiale
n, p = 20, 0.3
x_binom = np.arange(0, n+1)
pmf_binom = stats.binom.pmf(x_binom, n, p)
axes[0, 1].bar(x_binom, pmf_binom, color='green', alpha=0.7, edgecolor='black')
axes[0, 1].set_title(f'Binomiale(n={n}, p={p})\nE[X]={n*p:.1f}, σ={np.sqrt(n*p*(1-p)):.2f}', 
                     fontweight='bold')
axes[0, 1].set_xlabel('x')
axes[0, 1].set_ylabel('P(X=x)')
axes[0, 1].grid(axis='y', alpha=0.3)

# 3. Poisson
lambda_param = 5
x_poisson = np.arange(0, 20)
pmf_poisson = stats.poisson.pmf(x_poisson, lambda_param)
axes[0, 2].bar(x_poisson, pmf_poisson, color='orange', alpha=0.7, edgecolor='black')
axes[0, 2].set_title(f'Poisson(λ={lambda_param})\nE[X]=Var(X)={lambda_param}', 
                     fontweight='bold')
axes[0, 2].set_xlabel('x')
axes[0, 2].set_ylabel('P(X=x)')
axes[0, 2].grid(axis='y', alpha=0.3)

# 4. Normale standard
x_norm = np.linspace(-4, 4, 200)
pdf_norm = stats.norm.pdf(x_norm, 0, 1)
axes[1, 0].plot(x_norm, pdf_norm, 'b-', linewidth=2, label='PDF')
axes[1, 0].fill_between(x_norm, pdf_norm, alpha=0.3)

# Règle 68-95-99.7
axes[1, 0].axvline(-1, color='red', linestyle='--', alpha=0.5, label='μ±σ (68%)')
axes[1, 0].axvline(1, color='red', linestyle='--', alpha=0.5)
axes[1, 0].axvline(-2, color='orange', linestyle='--', alpha=0.5, label='μ±2σ (95%)')
axes[1, 0].axvline(2, color='orange', linestyle='--', alpha=0.5)

axes[1, 0].set_title('Normale(μ=0, σ²=1)', fontweight='bold')
axes[1, 0].set_xlabel('x')
axes[1, 0].set_ylabel('f(x)')
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3)

# 5. Comparaison de normales
params = [(0, 1), (0, 2), (2, 1)]
colors = ['blue', 'red', 'green']
for (mu, sigma), color in zip(params, colors):
    pdf = stats.norm.pdf(x_norm, mu, sigma)
    axes[1, 1].plot(x_norm, pdf, color=color, linewidth=2, 
                    label=f'μ={mu}, σ={sigma}')

axes[1, 1].set_title('Effet de μ et σ', fontweight='bold')
axes[1, 1].set_xlabel('x')
axes[1, 1].set_ylabel('f(x)')
axes[1, 1].legend()
axes[1, 1].grid(alpha=0.3)

# 6. Q-Q plot (Normal)
sample = np.random.normal(0, 1, 1000)
stats.probplot(sample, dist="norm", plot=axes[1, 2])
axes[1, 2].set_title('Q-Q Plot (test de normalité)', fontweight='bold')
axes[1, 2].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 🤖 APPLICATION ML : Échantillonnage et génération de données
print("🤖 ESSENTIEL EN ML : Génération de données synthétiques\n")

# 1. Classification binaire avec données gaussiennes
np.random.seed(42)
n_samples = 200

# Classe 0 : N(μ₀, Σ₀)
mean_0 = np.array([0, 0])
cov_0 = np.array([[1, 0.5], [0.5, 1]])
X_0 = np.random.multivariate_normal(mean_0, cov_0, n_samples)

# Classe 1 : N(μ₁, Σ₁)
mean_1 = np.array([3, 3])
cov_1 = np.array([[1, -0.3], [-0.3, 1]])
X_1 = np.random.multivariate_normal(mean_1, cov_1, n_samples)

# Visualisation
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.scatter(X_0[:, 0], X_0[:, 1], alpha=0.6, label='Classe 0', s=30)
ax1.scatter(X_1[:, 0], X_1[:, 1], alpha=0.6, label='Classe 1', s=30)
ax1.set_xlabel('Feature 1')
ax1.set_ylabel('Feature 2')
ax1.set_title('Dataset de classification généré', fontweight='bold')
ax1.legend()
ax1.grid(alpha=0.3)
ax1.axis('equal')

# 2. Histogrammes marginaux
ax2.hist(X_0[:, 0], bins=30, alpha=0.5, label='Classe 0 - Feature 1', density=True)
ax2.hist(X_1[:, 0], bins=30, alpha=0.5, label='Classe 1 - Feature 1', density=True)

# Overlay des densités théoriques
x_range = np.linspace(-3, 6, 200)
ax2.plot(x_range, stats.norm.pdf(x_range, mean_0[0], np.sqrt(cov_0[0, 0])), 
         'b-', linewidth=2, label='Théorique Classe 0')
ax2.plot(x_range, stats.norm.pdf(x_range, mean_1[0], np.sqrt(cov_1[0, 0])), 
         'r-', linewidth=2, label='Théorique Classe 1')

ax2.set_xlabel('Feature 1')
ax2.set_ylabel('Densité')
ax2.set_title('Distributions marginales', fontweight='bold')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("💡 Applications en ML :")
print("  • Génération de datasets synthétiques pour tests")
print("  • Augmentation de données (data augmentation)")
print("  • GANs (Generative Adversarial Networks)")
print("  • VAEs (Variational Autoencoders)")
print("  • Simulation de Monte Carlo")

## 6️⃣ Espérance, Variance et Moments

### 📐 Espérance mathématique

L'**espérance** $E[X]$ est la "moyenne pondérée" des valeurs.

**Cas discret** :
$$E[X] = \sum_{x} x \cdot P(X = x)$$

**Cas continu** :
$$E[X] = \int_{-\infty}^{\infty} x \cdot f_X(x) dx$$

**Propriétés** :
- Linéarité : $E[aX + b] = aE[X] + b$
- Additivité : $E[X + Y] = E[X] + E[Y]$ (toujours !)
- Si $X, Y$ indépendantes : $E[XY] = E[X]E[Y]$

### 📊 Variance

La **variance** mesure la dispersion autour de la moyenne.

$$\text{Var}(X) = E[(X - E[X])^2] = E[X^2] - (E[X])^2$$

**Écart-type** :
$$\sigma_X = \sqrt{\text{Var}(X)}$$

**Propriétés** :
- $\text{Var}(aX + b) = a^2 \text{Var}(X)$
- Si $X, Y$ indépendantes : $\text{Var}(X + Y) = \text{Var}(X) + \text{Var}(Y)$

### 📈 Moments d'ordre supérieur

**Moment d'ordre $n$** :
$$E[X^n]$$

**Moment centré d'ordre $n$** :
$$E[(X - E[X])^n]$$

**Skewness (asymétrie)** :
$$\gamma_1 = \frac{E[(X - \mu)^3]}{\sigma^3}$$

**Kurtosis (aplatissement)** :
$$\gamma_2 = \frac{E[(X - \mu)^4]}{\sigma^4}$$

In [ ]:
# Calcul et visualisation des moments
def calculer_moments(data: np.ndarray) -> dict:
    """Calcule les moments statistiques d'un échantillon"""
    return {
        'mean': np.mean(data),
        'variance': np.var(data, ddof=1),  # ddof=1 pour variance non biaisée
        'std': np.std(data, ddof=1),
        'skewness': stats.skew(data),
        'kurtosis': stats.kurtosis(data)
    }

# Génération de différentes distributions
np.random.seed(42)
n = 10000

distributions = {
    'Normal(0,1)': np.random.normal(0, 1, n),
    'Exponentielle(1)': np.random.exponential(1, n),
    'Uniforme(-2,2)': np.random.uniform(-2, 2, n),
    'Chi²(5)': np.random.chisquare(5, n)
}

# Visualisation
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.ravel()

for i, (name, data) in enumerate(distributions.items()):
    moments = calculer_moments(data)
    
    # Histogramme
    axes[i].hist(data, bins=50, density=True, alpha=0.7, 
                edgecolor='black', color='steelblue')
    
    # Ligne verticale pour la moyenne
    axes[i].axvline(moments['mean'], color='red', linestyle='--', 
                   linewidth=2, label=f"μ = {moments['mean']:.2f}")
    
    # Zone μ ± σ
    axes[i].axvspan(moments['mean'] - moments['std'], 
                   moments['mean'] + moments['std'], 
                   alpha=0.2, color='green', label=f"μ±σ ({moments['std']:.2f})")
    
    # Annotations
    textstr = f"Skewness: {moments['skewness']:.2f}\nKurtosis: {moments['kurtosis']:.2f}"
    axes[i].text(0.65, 0.95, textstr, transform=axes[i].transAxes,
                verticalalignment='top', bbox=dict(boxstyle='round', 
                facecolor='wheat', alpha=0.5))
    
    axes[i].set_title(name, fontweight='bold')
    axes[i].set_xlabel('Valeur')
    axes[i].set_ylabel('Densité')
    axes[i].legend()
    axes[i].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Tableau récapitulatif
print("\n📊 Tableau des moments :\n")
df_moments = pd.DataFrame({
    name: calculer_moments(data) 
    for name, data in distributions.items()
}).T

print(df_moments.round(3))

print("\n💡 Interprétation :")
print("  • Skewness > 0 : Queue à droite (asymétrie positive)")
print("  • Skewness < 0 : Queue à gauche (asymétrie négative)")
print("  • Kurtosis > 0 : Plus 'pointue' que normale (queues lourdes)")
print("  • Kurtosis < 0 : Plus 'plate' que normale (queues légères)")

## 7️⃣ Covariance et Corrélation

### 🔗 Covariance

La **covariance** mesure la relation linéaire entre deux variables.

$$\text{Cov}(X, Y) = E[(X - E[X])(Y - E[Y])] = E[XY] - E[X]E[Y]$$

**Propriétés** :
- $\text{Cov}(X, X) = \text{Var}(X)$
- $\text{Cov}(X, Y) = \text{Cov}(Y, X)$ (symétrie)
- $\text{Cov}(aX + b, Y) = a\text{Cov}(X, Y)$
- Si $X, Y$ indépendantes : $\text{Cov}(X, Y) = 0$ (réciproque fausse !)

**Variance d'une somme** :
$$\text{Var}(X + Y) = \text{Var}(X) + \text{Var}(Y) + 2\text{Cov}(X, Y)$$

### 📐 Corrélation (de Pearson)

La **corrélation** est la covariance normalisée :

$$\rho(X, Y) = \frac{\text{Cov}(X, Y)}{\sigma_X \sigma_Y}$$

**Propriétés** :
- $-1 \leq \rho \leq 1$
- $\rho = 1$ : corrélation linéaire parfaite positive
- $\rho = -1$ : corrélation linéaire parfaite négative
- $\rho = 0$ : pas de corrélation linéaire (⚠️ peut y avoir dépendance non-linéaire !)

### 📊 Matrice de covariance

Pour un vecteur aléatoire $\mathbf{X} = (X_1, ..., X_n)$ :

$$\Sigma = \begin{pmatrix}
\text{Var}(X_1) & \text{Cov}(X_1, X_2) & \cdots & \text{Cov}(X_1, X_n) \\
\text{Cov}(X_2, X_1) & \text{Var}(X_2) & \cdots & \text{Cov}(X_2, X_n) \\
\vdots & \vdots & \ddots & \vdots \\
\text{Cov}(X_n, X_1) & \text{Cov}(X_n, X_2) & \cdots & \text{Var}(X_n)
\end{pmatrix}$$

**ESSENTIEL EN ML** : PCA, Gaussian Processes, portfolio optimization !

In [ ]:
# Génération de données corrélées
def generer_donnees_correlees(n: int, rho: float) -> Tuple[np.ndarray, np.ndarray]:
    """
    Génère deux variables corrélées avec coefficient ρ
    """
    # Matrice de covariance
    cov = np.array([[1, rho], [rho, 1]])
    
    # Génération
    data = np.random.multivariate_normal([0, 0], cov, n)
    return data[:, 0], data[:, 1]

# Visualisation pour différentes corrélations
correlations = [-0.9, -0.5, 0, 0.5, 0.9]
fig, axes = plt.subplots(2, 3, figsize=(15, 9))
axes = axes.ravel()

np.random.seed(42)
n = 500

for i, rho in enumerate(correlations):
    X, Y = generer_donnees_correlees(n, rho)
    
    # Calcul de la corrélation empirique
    rho_empirique = np.corrcoef(X, Y)[0, 1]
    cov_empirique = np.cov(X, Y, ddof=1)[0, 1]
    
    # Scatter plot
    axes[i].scatter(X, Y, alpha=0.5, s=20)
    
    # Régression linéaire
    z = np.polyfit(X, Y, 1)
    p = np.poly1d(z)
    x_line = np.linspace(X.min(), X.max(), 100)
    axes[i].plot(x_line, p(x_line), 'r-', linewidth=2, alpha=0.8)
    
    axes[i].set_title(f'ρ théorique = {rho:.1f}\nρ empirique = {rho_empirique:.3f}',
                     fontweight='bold')
    axes[i].set_xlabel('X')
    axes[i].set_ylabel('Y')
    axes[i].grid(alpha=0.3)
    axes[i].axis('equal')
    
    # Annotation de la covariance
    axes[i].text(0.05, 0.95, f'Cov(X,Y) = {cov_empirique:.3f}',
                transform=axes[i].transAxes, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# Cas non-linéaire (dernière subplot)
X = np.random.uniform(-3, 3, n)
Y = X**2 + np.random.normal(0, 1, n)
rho_nonlinear = np.corrcoef(X, Y)[0, 1]

axes[5].scatter(X, Y, alpha=0.5, s=20, color='purple')
axes[5].set_title(f'Dépendance NON-linéaire\nρ = {rho_nonlinear:.3f} ≈ 0 !',
                 fontweight='bold', color='red')
axes[5].set_xlabel('X')
axes[5].set_ylabel('Y = X² + bruit')
axes[5].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n⚠️ ATTENTION :")
print("  • Corrélation = 0 ≠ Indépendance !")
print("  • La corrélation mesure UNIQUEMENT les relations linéaires")
print("  • Pour dépendances non-linéaires : mutual information, rank correlation, etc.")

In [ ]:
# 📈 APPLICATION FINANCE : Matrice de corrélation d'un portefeuille
print("📈 APPLICATION FINANCE : Analyse de corrélation de portefeuille\n")

# Simulation de rendements d'actifs
np.random.seed(42)
n_days = 252  # 1 an de trading
n_assets = 5

# Matrice de corrélation réaliste
rho_matrix = np.array([
    [1.0, 0.7, 0.5, 0.3, 0.1],   # Tech stock
    [0.7, 1.0, 0.6, 0.4, 0.2],   # Tech stock 2
    [0.5, 0.6, 1.0, 0.2, 0.0],   # Consumer goods
    [0.3, 0.4, 0.2, 1.0, -0.3],  # Energy
    [0.1, 0.2, 0.0, -0.3, 1.0]   # Gold (safe haven)
])

# Volatilités annuelles
volatilities = np.array([0.25, 0.30, 0.20, 0.35, 0.15])  # 15-35% par an

# Matrice de covariance (annualisée)
cov_matrix = np.outer(volatilities, volatilities) * rho_matrix

# Simulation des rendements quotidiens
mean_returns = np.array([0.10, 0.12, 0.08, 0.15, 0.05]) / 252  # Rendements annuels → quotidiens
cov_daily = cov_matrix / 252  # Variance annuelle → quotidienne

returns = np.random.multivariate_normal(mean_returns, cov_daily, n_days)

asset_names = ['Tech A', 'Tech B', 'Consumer', 'Energy', 'Gold']

# Visualisation de la matrice de corrélation
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Heatmap de corrélation
im = ax1.imshow(rho_matrix, cmap='RdYlGn', vmin=-1, vmax=1, aspect='auto')
ax1.set_xticks(range(n_assets))
ax1.set_yticks(range(n_assets))
ax1.set_xticklabels(asset_names, rotation=45, ha='right')
ax1.set_yticklabels(asset_names)
ax1.set_title('Matrice de Corrélation', fontweight='bold')

# Annotations
for i in range(n_assets):
    for j in range(n_assets):
        text = ax1.text(j, i, f'{rho_matrix[i, j]:.2f}',
                       ha="center", va="center", color="black", fontweight='bold')

plt.colorbar(im, ax=ax1, label='Corrélation')

# Évolution des rendements cumulés
cumulative_returns = (1 + returns).cumprod(axis=0)
for i, name in enumerate(asset_names):
    ax2.plot(cumulative_returns[:, i], label=name, linewidth=2)

ax2.set_xlabel('Jours de trading')
ax2.set_ylabel('Valeur du portefeuille (base 1.0)')
ax2.set_title('Évolution des actifs', fontweight='bold')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Calcul du risque de portefeuille
weights = np.array([0.3, 0.2, 0.2, 0.2, 0.1])  # Répartition du portefeuille

# Variance du portefeuille : w^T Σ w
portfolio_variance = weights @ cov_matrix @ weights
portfolio_volatility = np.sqrt(portfolio_variance)

# Rendement attendu du portefeuille
annual_returns = np.array([0.10, 0.12, 0.08, 0.15, 0.05])
portfolio_return = weights @ annual_returns

print(f"\n📊 Analyse du portefeuille :")
print(f"\nRépartition : {dict(zip(asset_names, weights))}")
print(f"\nRendement attendu : {portfolio_return*100:.2f}% par an")
print(f"Volatilité (risque) : {portfolio_volatility*100:.2f}% par an")
print(f"Ratio de Sharpe (simplifié) : {portfolio_return/portfolio_volatility:.2f}")

print("\n💡 Effet de diversification :")
print(f"  • Gold a corrélation négative avec Energy (-0.3)")
print(f"  • ⇒ Réduit le risque global du portefeuille")
print(f"  • Tech A et Tech B très corrélées (0.7) ⇒ Moins de diversification")

## 🎯 Résumé du Chapitre

### ✅ Concepts clés maîtrisés

1. **Espace de probabilité** : (Ω, F, P)
2. **Probabilités conditionnelles** : P(A|B) = P(A∩B) / P(B)
3. **Théorème de Bayes** : P(A|B) = P(B|A)P(A) / P(B)
4. **Variables aléatoires** : Discrètes (PMF) et continues (PDF)
5. **Distributions classiques** :
   - Bernoulli, Binomiale, Poisson
   - **Normale** (la plus importante !)
6. **Moments** : Espérance, variance, skewness, kurtosis
7. **Corrélation** : Mesure de dépendance linéaire

### 🤖 Applications en ML et Finance

| Concept | ML Application | Finance Application |
|---------|---------------|---------------------|
| Bayes | Naive Bayes, Bayesian ML | Filtrage de signaux |
| Normale | Initialisation poids, erreurs | Rendements, Black-Scholes |
| Covariance | PCA, feature correlation | Diversification, risque |
| Échantillonnage | Data augmentation, GANs | Monte Carlo, simulation |
| Moments | Feature engineering | Distribution analysis |

### 📚 Pour aller plus loin

- ✅ **Exercices** : [exercices_07_probabilites.ipynb](../envs/phase_1_math/exercices_07_probabilites.ipynb)
- ✅ **Projet** : [projet_07_monte_carlo.ipynb](../envs/phase_1_math/projet_07_monte_carlo.ipynb)
- 📖 **Chapitre suivant** : Statistiques inférentielles

---

### 🎓 Points essentiels à retenir

1. 🌟 **Théorème de Bayes = cœur du ML bayésien**
2. 📊 **Loi normale = distribution fondamentale**
3. 🔗 **Corrélation ≠ Causalité** (et corrélation = 0 ≠ indépendance)
4. 🎲 **Échantillonnage = génération de données synthétiques**
5. 📈 **Matrice de covariance = clé de la diversification**